In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import utils
import constants
import chess as pychess
import numpy as np
import os
import random

Data files: 1 - 27
 - each has 1 000 000 rows
 - treat 1 - 25 as training data
 - 26 as final test data
 - 27 as validation data after training is completed (for preliminary comparisons)

In [2]:
## Split the data into chunks
def split_data():
    years = ["2016", "2017", "2018", "2019", "2020", "2021"]
    size = 1000000
    buffer = pd.DataFrame([], columns=["board", "move"])
    split_counter = 1

    if not os.path.exists("data/split/"):
        os.mkdir("data/split/")

    for year in years:
        big = pd.read_csv(f"data/chess{year}.csv").drop('Unnamed: 0', axis=1)
        i = 0
        while i<len(big):
            nrows = size - len(buffer)
            buffer = pd.concat([buffer, big.iloc[i:(i+nrows-1), :]])
            i += nrows
            if len(buffer) == size:
                buffer.to_csv(f"data/split/chess{split_counter}.csv", index=False)
                split_counter += 1
                buffer = pd.DataFrame([], columns=["board", "move"])
    if len(buffer) > 0:
        buffer.to_csv(f"data/split/chess{split_counter}.csv", index=False)

# split_data()

In [3]:
def restrict_legal(move_arr, fen):
    ''' Restricts the move probability array to only legal moves and rescale remaining legal move probabilities '''
    board = pychess.Board(fen)
    legal_arr = np.zeros(constants.LEN_UCI_MOVES, dtype=np.int16)
    for move in board.legal_moves:
        legal_arr[constants.uci_moves[move.uci()]] = 1
    for i in range(constants.LEN_UCI_MOVES):
        if legal_arr[i] == 0:
            move_arr[i] = 0
    return move_arr / np.sum(move_arr)

In [28]:
def train_model(model, data_dir="data/split/", n_files=25, epochs=1, ckpt_dir="ckpt/"):

    file_order = list(range(1, n_files+1))

    if not os.path.exists(ckpt_dir):
        os.mkdir(ckpt_dir)
    if not os.path.exists(utils.path_join(ckpt_dir, model.name)):
        os.mkdir(utils.path_join(ckpt_dir, model.name))

    checkpoints = [utils.path_join(ckpt_dir, model.name, name) for name in os.listdir(utils.path_join(ckpt_dir, model.name))]
    if checkpoints:
        latest_checkpoint = max(checkpoints, key=os.path.getctime)
        print("Restoring from", latest_checkpoint)
        model = keras.models.load_model(latest_checkpoint)
        n = latest_checkpoint.split("-")[-1][0]
    else:
        n = 0

    processed_files = [filename.split(".")[0] for filename in os.listdir(utils.path_join(data_dir, "processed"))]
    for file in file_order:
        if file <= int(n):
            continue
        filename = f"chess{file}"
        print(f"Training on file {filename}")

        # load data from .csv file
        if filename in processed_files:
            df = pd.read_pickle(utils.path_join(data_dir, "processed", filename + ".pkl"))
        else:
            df = pd.read_csv(utils.path_join(data_dir, filename + ".csv"))

            # translate from fen to obs arr
            print("Processing...", end="")
            df.loc[:, "obs"] = df.loc[:, "board"].map(utils.parse_fen)
            print("complete")

        # convert to numpy arrays
        x = np.array(df["obs"].values.tolist())
        y = df["move"].values

        # train
        model.fit(x, y, batch_size=64, epochs=epochs, validation_split=0.1)

        # save checkpoint between files
        model.save(utils.path_join(ckpt_dir, model.name, f"{model.name}-{file}.h5"))
    print(f"Training completed. Final save file: {utils.path_join(ckpt_dir, model.name, f'{model.name}-{file}.h5')}")

In [29]:
def evaluate_model(model_names, data="validation", data_dir="data/split/", ckpt_dir="ckpt/", metric="sparse_categorical_accuracy"):

    models = []
    for model_name in model_names:
        checkpoints = [utils.path_join(ckpt_dir, model_name, name) for name in os.listdir(utils.path_join(ckpt_dir, model_name))]
        if checkpoints:
            latest_checkpoint = max(checkpoints, key=os.path.getctime)
            print("Restoring from", latest_checkpoint)
            model = keras.models.load_model(latest_checkpoint)
            models.append(model)
        else:
            print("No checkpoint available")

    # load data
    if data == "validation":
        data = pd.read_csv(utils.path_join(data_dir, "chess27.csv"))
    elif data == "test":
        data = pd.read_csv(utils.path_join(data_dir, "chess26.csv"))
    
    # convert fen to obs
    print("Processing...", end="")
    data.loc[:, "obs"] = data.loc[:, "board"].map(utils.parse_fen)
    print("complete")

    # convert to numpy arrays
    x = np.array(data["obs"].values.tolist())
    y = data["move"].values

    performance = []
    for model_name, model in zip(model_names, models):
        performance.append([model_name, model.evaluate(x, y, batch_size=128, return_dict=True)[metric]])

    return pd.DataFrame(performance, columns=["model_name", metric])

In [6]:
# model base
activation = "relu"
model = keras.Sequential([
    keras.layers.Input(shape=(70)),
    keras.layers.Dense(210, activation=activation),
    keras.layers.Dense(630, activation=activation),
    keras.layers.Dense(210, activation=activation),
    keras.layers.Dense(1200, activation=activation),
    keras.layers.Dense(constants.LEN_UCI_MOVES, activation="softmax")
], name="dense1")

model.summary()

model.compile(
    optimizer = keras.optimizers.Adam(learning_rate=0.001),
    loss = keras.losses.SparseCategoricalCrossentropy(),
    metrics = [keras.metrics.SparseCategoricalAccuracy()],
)

Model: "dense1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 210)               14910     
                                                                 
 dense_1 (Dense)             (None, 630)               132930    
                                                                 
 dense_2 (Dense)             (None, 210)               132510    
                                                                 
 dense_3 (Dense)             (None, 1200)              253200    
                                                                 
 dense_4 (Dense)             (None, 1968)              2363568   
                                                                 
Total params: 2,897,118
Trainable params: 2,897,118
Non-trainable params: 0
_________________________________________________________________


In [26]:
# model base with dropout
activation = "relu"
model = keras.Sequential([
    keras.layers.Input(shape=(70)),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(630, activation=activation),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(630, activation=activation),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(630, activation=activation),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1200, activation=activation),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(constants.LEN_UCI_MOVES, activation="softmax")
], name="dense_drop")

model.summary()

model.compile(
    optimizer = keras.optimizers.Adam(learning_rate=0.001),
    loss = keras.losses.SparseCategoricalCrossentropy(),
    metrics = [keras.metrics.SparseCategoricalAccuracy()],
)

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout (Dropout)           (None, 70)                0         
                                                                 
 dense_37 (Dense)            (None, 630)               44730     
                                                                 
 dropout_1 (Dropout)         (None, 630)               0         
                                                                 
 dense_38 (Dense)            (None, 630)               397530    
                                                                 
 dropout_2 (Dropout)         (None, 630)               0         
                                                                 
 dense_39 (Dense)            (None, 630)               397530    
                                                                 
 dropout_3 (Dropout)         (None, 630)             

In [32]:
# model deep
activation = "relu"
model = keras.Sequential([
    keras.layers.Input(shape=(70)),
], name="dense_deep")
for _ in range(16):
    model.add(keras.layers.Dense(256, activation=activation))
model.add(keras.layers.Dense(constants.LEN_UCI_MOVES, activation="softmax"))

model.summary()

model.compile(
    optimizer = keras.optimizers.Adam(learning_rate=0.001),
    loss = keras.losses.SparseCategoricalCrossentropy(),
    metrics = [keras.metrics.SparseCategoricalAccuracy()],
)

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_75 (Dense)            (None, 256)               18176     
                                                                 
 dense_76 (Dense)            (None, 256)               65792     
                                                                 
 dense_77 (Dense)            (None, 256)               65792     
                                                                 
 dense_78 (Dense)            (None, 256)               65792     
                                                                 
 dense_79 (Dense)            (None, 256)               65792     
                                                                 
 dense_80 (Dense)            (None, 256)               65792     
                                                                 
 dense_81 (Dense)            (None, 256)             

In [ ]:
# model deep with dropout
activation = "relu"
model = keras.Sequential([
    keras.layers.Input(shape=(70)),
], name="dense_deep_drop")
for _ in range(16):
    model.add(keras.layers.Dense(256, activation=activation))
    model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(constants.LEN_UCI_MOVES, activation="softmax"))

model.summary()

model.compile(
    optimizer = keras.optimizers.Adam(learning_rate=0.001),
    loss = keras.losses.SparseCategoricalCrossentropy(),
    metrics = [keras.metrics.SparseCategoricalAccuracy()],
)

In [9]:
# model conv2d (semi) sequential
activation = "relu"

inputs = keras.Input(shape=(70), name="all_input")

input1 = keras.layers.Lambda(lambda x: tf.slice(x, (0, 0), (-1, 1)), name="player_input")(inputs)
input2 = keras.layers.Lambda(lambda x: tf.slice(x, (0, 1), (-1, 64)), name="board_input")(inputs)
input3 = keras.layers.Lambda(lambda x: tf.slice(x, (0, 65), (-1, 5)), name="misc_input")(inputs)

conv_model = tf.keras.models.Sequential([  # 1:65
    keras.layers.Reshape((8, 8, 1), input_shape=(64,)),
    keras.layers.Conv2D(64, 3, padding="same", activation=activation),
    keras.layers.Conv2D(64, 3, padding="same", activation=activation),
    keras.layers.Conv2D(64, 3, padding="same", activation=activation),
    keras.layers.Flatten(),
    keras.layers.Dense(128)
], name="conv_model")
conv_output = conv_model(input2)

dense_inputs = keras.layers.concatenate([input1, conv_output, input3])
dense_model = tf.keras.models.Sequential([  # tf.concat([input[:1], conv_model_output, input[65:]], axis=0)
    keras.layers.Input(134),
    keras.layers.Dense(256, activation=activation),
    keras.layers.Dense(256, activation=activation),
    keras.layers.Dense(constants.LEN_UCI_MOVES, activation="softmax")
], name="dense_model")
dense_output = dense_model(dense_inputs)

model = tf.keras.models.Model(inputs, dense_output, name="conv2d_seq")
model.summary()

model.compile(
    optimizer = keras.optimizers.Adam(learning_rate=0.001),
    loss = keras.losses.SparseCategoricalCrossentropy(),
    metrics = [keras.metrics.SparseCategoricalAccuracy()],
)

'''
input2  ->  conv_model
input1 + conv_model_output + input3 -> dense_model
'''

Model: "conv2d_seq"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 all_input (InputLayer)         [(None, 70)]         0           []                               
                                                                                                  
 board_input (Lambda)           (None, 64)           0           ['all_input[0][0]']              
                                                                                                  
 player_input (Lambda)          (None, 1)            0           ['all_input[0][0]']              
                                                                                                  
 conv_model (Sequential)        (None, 128)          598912      ['board_input[0][0]']            
                                                                                         

'\ninput2  ->  conv_model\ninput1 + conv_model_output + input3 -> dense_model\n'

In [10]:
train_model(model, n_files=5)

Restoring from ckpt/conv2d_seq/conv2d_seq-2.h5
Training on file chess3.csv
Processing...complete
14063/14063 [==============================] - 1392s 99ms/step - loss: 3.8036 - sparse_categorical_accuracy: 0.1914 - val_loss: 3.7828 - val_sparse_categorical_accuracy: 0.1876
Training on file chess4.csv
Processing...complete
14063/14063 [==============================] - 1036s 74ms/step - loss: 3.6329 - sparse_categorical_accuracy: 0.2037 - val_loss: 3.5467 - val_sparse_categorical_accuracy: 0.2087
Training on file chess5.csv
Processing...complete
14063/14063 [==============================] - 943s 67ms/step - loss: 3.5195 - sparse_categorical_accuracy: 0.2127 - val_loss: 3.5112 - val_sparse_categorical_accuracy: 0.2096
Training completed. Final save file: ckpt/conv2d_seq/conv2d_seq-5.h5


In [35]:
# model concurrent conv2d
activation = "relu"

inputs = keras.Input(shape=(70), name="all_input")
input1 = keras.layers.Lambda(lambda x: tf.slice(x, (0, 0), (-1, 1)), name="player_input")(inputs)
input2 = keras.layers.Lambda(lambda x: tf.slice(x, (0, 1), (-1, 64)), name="board_input")(inputs)
input3 = keras.layers.Lambda(lambda x: tf.slice(x, (0, 65), (-1, 5)), name="misc_input")(inputs)

conv3_model = tf.keras.models.Sequential([  # 1:65
    keras.layers.Reshape((8, 8, 1), input_shape=(64,)),
    keras.layers.Conv2D(32, 3, padding="same", activation=activation),
    keras.layers.Conv2D(32, 3, padding="same", activation=activation),
    keras.layers.Flatten(),
    keras.layers.Dense(64)
], name="conv_model_k3")
conv3_output = conv3_model(input2)

conv5_model = tf.keras.models.Sequential([  # 1:65
    keras.layers.Reshape((8, 8, 1), input_shape=(64,)),
    keras.layers.Conv2D(32, 5, padding="same", activation=activation),
    keras.layers.Conv2D(32, 5, padding="same", activation=activation),
    keras.layers.Flatten(),
    keras.layers.Dense(64)
], name="conv_model_k5")
conv5_output = conv5_model(input2)

conv7_model = tf.keras.models.Sequential([  # 1:65
    keras.layers.Reshape((8, 8, 1), input_shape=(64,)),
    keras.layers.Conv2D(32, 7, padding="same", activation=activation),
    keras.layers.Conv2D(32, 7, padding="same", activation=activation),
    keras.layers.Flatten(),
    keras.layers.Dense(64)
], name="conv_model_k7")
conv7_output = conv7_model(input2)

dense_inputs = keras.layers.concatenate([input1, conv3_output, conv5_output, conv7_output, input3])
dense_model = tf.keras.models.Sequential([  # tf.concat([input[:1], conv_model_output, input[65:]], axis=0)
    keras.layers.Input(198),
    keras.layers.Dense(256, activation=activation),
    keras.layers.Dense(256, activation=activation),
    keras.layers.Dense(constants.LEN_UCI_MOVES, activation="softmax")
], name="dense_model")
dense_output = dense_model(dense_inputs)

model = tf.keras.models.Model(inputs, dense_output, name="conv2d_357")
model.summary()

model.compile(
    optimizer = keras.optimizers.Adam(learning_rate=0.001),
    loss = keras.losses.SparseCategoricalCrossentropy(),
    metrics = [keras.metrics.SparseCategoricalAccuracy()],
)

'''
input2  ->  conv3_model (kernel size 3)   - all 2 layers
input2  ->  conv5_model (kernel size 5)
input2  ->  conv7_model (kernel size 7)

input1 + conv3 + conv5 + conv7 + input3 -> dense_model
'''

Model: "conv2d_357"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 all_input (InputLayer)         [(None, 70)]         0           []                               
                                                                                                  
 board_input (Lambda)           (None, 64)           0           ['all_input[0][0]']              
                                                                                                  
 player_input (Lambda)          (None, 1)            0           ['all_input[0][0]']              
                                                                                                  
 conv_model_k3 (Sequential)     (None, 64)           140704      ['board_input[0][0]']            
                                                                                         

'\ninput2  ->  conv3_model (kernel size 3)   - all 2 layers\ninput2  ->  conv5_model (kernel size 5)\ninput2  ->  conv7_model (kernel size 7)\n\ninput1 + conv3 + conv5 + conv7 + input3 -> dense_model\n'

In [36]:
# train_model(model, n_files=5)

Restoring from ckpt/conv2d_357/conv2d_357-3.h5
Training on file chess4.csv
Processing...complete
14063/14063 [==============================] - 1351s 95ms/step - loss: 3.7146 - sparse_categorical_accuracy: 0.1946 - val_loss: 3.6616 - val_sparse_categorical_accuracy: 0.1958
Training on file chess5.csv
Processing...complete
14063/14063 [==============================] - 1116s 79ms/step - loss: 3.6199 - sparse_categorical_accuracy: 0.2022 - val_loss: 3.6087 - val_sparse_categorical_accuracy: 0.2024
Training completed. Final save file: ckpt/conv2d_357/conv2d_357-5.h5


In [6]:
# model conv2d with kernel size 3 and 5
activation = "relu"

inputs = keras.Input(shape=(70), name="all_input")
input1 = keras.layers.Lambda(lambda x: tf.slice(x, (0, 0), (-1, 1)), name="player_input")(inputs)
input2 = keras.layers.Lambda(lambda x: tf.slice(x, (0, 1), (-1, 64)), name="board_input")(inputs)
input3 = keras.layers.Lambda(lambda x: tf.slice(x, (0, 65), (-1, 5)), name="misc_input")(inputs)

conv3_model = tf.keras.models.Sequential([  # 1:65
    keras.layers.Reshape((8, 8, 1), input_shape=(64,)),
    keras.layers.Conv2D(32, 3, padding="same", activation=activation),
    keras.layers.Conv2D(32, 3, padding="same", activation=activation),
    keras.layers.Conv2D(32, 3, padding="same", activation=activation),
    keras.layers.Flatten(),
    keras.layers.Dense(128)
], name="conv_model_k3")
conv3_output = conv3_model(input2)

conv5_model = tf.keras.models.Sequential([  # 1:65
    keras.layers.Reshape((8, 8, 1), input_shape=(64,)),
    keras.layers.Conv2D(32, 5, padding="same", activation=activation),
    keras.layers.Conv2D(32, 5, padding="same", activation=activation),
    keras.layers.Conv2D(32, 5, padding="same", activation=activation),
    keras.layers.Flatten(),
    keras.layers.Dense(128)
], name="conv_model_k5")
conv5_output = conv5_model(input2)

dense_inputs = keras.layers.concatenate([input1, conv3_output, conv5_output, input3])
dense_model = tf.keras.models.Sequential([  # tf.concat([input[:1], conv_model_output, input[65:]], axis=0)
    keras.layers.Input(262),
    keras.layers.Dense(256, activation=activation),
    keras.layers.Dense(256, activation=activation),
    keras.layers.Dense(256, activation=activation),
    keras.layers.Dense(constants.LEN_UCI_MOVES, activation="softmax")
], name="dense_model")
dense_output = dense_model(dense_inputs)

model = tf.keras.models.Model(inputs, dense_output, name="conv2d_35")
model.summary()

model.compile(
    optimizer = keras.optimizers.Adam(learning_rate=0.001),
    loss = keras.losses.SparseCategoricalCrossentropy(),
    metrics = [keras.metrics.SparseCategoricalAccuracy()],
)

'''
input2  ->  conv3_model (kernel size 3)   - all 3 layers
input2  ->  conv5_model (kernel size 5)

input1 + conv3 + conv5 + input3 -> dense_model
'''

Model: "conv2d_35"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 all_input (InputLayer)         [(None, 70)]         0           []                               
                                                                                                  
 board_input (Lambda)           (None, 64)           0           ['all_input[0][0]']              
                                                                                                  
 player_input (Lambda)          (None, 1)            0           ['all_input[0][0]']              
                                                                                                  
 conv_model_k3 (Sequential)     (None, 128)          281088      ['board_input[0][0]']            
                                                                                          

'\ninput2  ->  conv3_model (kernel size 3)   - all 3 layers\ninput2  ->  conv5_model (kernel size 5)\n\ninput1 + conv3 + conv5 + input3 -> dense_model\n'

In [7]:
train_model(model, n_files=5)

Restoring from ckpt/conv2d_35/conv2d_35-4.h5
Training on file chess5.csv
Processing...complete
14063/14063 [==============================] - 1255s 89ms/step - loss: 3.8425 - sparse_categorical_accuracy: 0.1910 - val_loss: 3.8298 - val_sparse_categorical_accuracy: 0.1919
Training completed. Final save file: ckpt/conv2d_35/conv2d_35-5.h5


In [12]:
# model conv2d with kernel size 3 and 5, expanded conv_models
activation = "relu"

inputs = keras.Input(shape=(70), name="all_input")
input1 = keras.layers.Lambda(lambda x: tf.slice(x, (0, 0), (-1, 1)), name="player_input")(inputs)
input2 = keras.layers.Lambda(lambda x: tf.slice(x, (0, 1), (-1, 64)), name="board_input")(inputs)
input3 = keras.layers.Lambda(lambda x: tf.slice(x, (0, 65), (-1, 5)), name="misc_input")(inputs)

conv3_model = tf.keras.models.Sequential([  # 1:65
    keras.layers.Reshape((8, 8, 1), input_shape=(64,)),
    keras.layers.Conv2D(64, 3, padding="same", activation=activation),
    keras.layers.Conv2D(64, 3, padding="same", activation=activation),
    keras.layers.Conv2D(64, 3, padding="same", activation=activation),
    keras.layers.Flatten(),
    keras.layers.Dense(64)
], name="conv_model_k3")
conv3_output = conv3_model(input2)

conv5_model = tf.keras.models.Sequential([  # 1:65
    keras.layers.Reshape((8, 8, 1), input_shape=(64,)),
    keras.layers.Conv2D(64, 5, padding="same", activation=activation),
    keras.layers.Conv2D(64, 5, padding="same", activation=activation),
    keras.layers.Conv2D(64, 5, padding="same", activation=activation),
    keras.layers.Flatten(),
    keras.layers.Dense(64)
], name="conv_model_k5")
conv5_output = conv5_model(input2)

dense_inputs = keras.layers.concatenate([input1, conv3_output, conv5_output, input3])
dense_model = tf.keras.models.Sequential([  # tf.concat([input[:1], conv_model_output, input[65:]], axis=0)
    keras.layers.Input(134),
    keras.layers.Dense(256, activation=activation),
    keras.layers.Dense(256, activation=activation),
    keras.layers.Dense(256, activation=activation),
    keras.layers.Dense(constants.LEN_UCI_MOVES, activation="softmax")
], name="dense_model")
dense_output = dense_model(dense_inputs)

model = tf.keras.models.Model(inputs, dense_output, name="conv2d_35x1")
model.summary()

model.compile(
    optimizer = keras.optimizers.Adam(learning_rate=0.001),
    loss = keras.losses.SparseCategoricalCrossentropy(),
    metrics = [keras.metrics.SparseCategoricalAccuracy()],
)

'''
input2  ->  conv3_model (kernel size 3)   - all 3 layers
input2  ->  conv5_model (kernel size 5)

input1 + conv3 + conv5 + conv7 + input3 -> dense_model
'''

Model: "conv2d_35x1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 all_input (InputLayer)         [(None, 70)]         0           []                               
                                                                                                  
 board_input (Lambda)           (None, 64)           0           ['all_input[0][0]']              
                                                                                                  
 player_input (Lambda)          (None, 1)            0           ['all_input[0][0]']              
                                                                                                  
 conv_model_k3 (Sequential)     (None, 64)           336704      ['board_input[0][0]']            
                                                                                        

'\ninput2  ->  conv3_model (kernel size 3)   - all 3 layers\ninput2  ->  conv5_model (kernel size 5)\n\ninput1 + conv3 + conv5 + conv7 + input3 -> dense_model\n'

In [13]:
train_model(model, n_files=5)

Training on file chess1.csv
Processing...complete


2022-09-14 12:05:44.487185: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


14063/14063 [==============================] - 462s 33ms/step - loss: 5.0803 - sparse_categorical_accuracy: 0.1147 - val_loss: 4.4830 - val_sparse_categorical_accuracy: 0.1578
Training on file chess2.csv
Processing...complete
14063/14063 [==============================] - 470s 33ms/step - loss: 4.3066 - sparse_categorical_accuracy: 0.1658 - val_loss: 4.2156 - val_sparse_categorical_accuracy: 0.1652
Training on file chess3.csv
Processing...complete
14063/14063 [==============================] - 444s 32ms/step - loss: 4.1095 - sparse_categorical_accuracy: 0.1746 - val_loss: 4.0781 - val_sparse_categorical_accuracy: 0.1715
Training on file chess4.csv
Processing...complete
14063/14063 [==============================] - 450s 32ms/step - loss: 3.9342 - sparse_categorical_accuracy: 0.1855 - val_loss: 3.8638 - val_sparse_categorical_accuracy: 0.1895
Training on file chess5.csv
Processing...complete
14063/14063 [==============================] - 453s 32ms/step - loss: 3.8185 - sparse_categorica

In [17]:
# model conv2d (semi) sequential expanded dense
activation = "relu"

inputs = keras.Input(shape=(70), name="all_input")

input1 = keras.layers.Lambda(lambda x: tf.slice(x, (0, 0), (-1, 1)), name="player_input")(inputs)
input2 = keras.layers.Lambda(lambda x: tf.slice(x, (0, 1), (-1, 64)), name="board_input")(inputs)
input3 = keras.layers.Lambda(lambda x: tf.slice(x, (0, 65), (-1, 5)), name="misc_input")(inputs)

conv_model = tf.keras.models.Sequential([  # 1:65
    keras.layers.Reshape((8, 8, 1), input_shape=(64,)),
    keras.layers.Conv2D(64, 3, padding="same", activation=activation),
    keras.layers.Conv2D(64, 3, padding="same", activation=activation),
    keras.layers.Conv2D(64, 3, padding="same", activation=activation),
    keras.layers.Flatten(),
    keras.layers.Dense(128)
], name="conv_model")
conv_output = conv_model(input2)

dense_inputs = keras.layers.concatenate([input1, conv_output, input3])
dense_model = tf.keras.models.Sequential([  # tf.concat([input[:1], conv_model_output, input[65:]], axis=0)
    keras.layers.Input(134),
    keras.layers.Dense(256, activation=activation),
    keras.layers.Dense(256, activation=activation),
    keras.layers.Dense(512, activation=activation),
    keras.layers.Dense(constants.LEN_UCI_MOVES, activation="softmax")
], name="dense_model")
dense_output = dense_model(dense_inputs)

model = tf.keras.models.Model(inputs, dense_output, name="conv2d_seq_dx")
model.summary()

model.compile(
    optimizer = keras.optimizers.Adam(learning_rate=0.001),
    loss = keras.losses.SparseCategoricalCrossentropy(),
    metrics = [keras.metrics.SparseCategoricalAccuracy()],
)

Model: "conv2d_seq_dx"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 all_input (InputLayer)         [(None, 70)]         0           []                               
                                                                                                  
 board_input (Lambda)           (None, 64)           0           ['all_input[0][0]']              
                                                                                                  
 player_input (Lambda)          (None, 1)            0           ['all_input[0][0]']              
                                                                                                  
 conv_model (Sequential)        (None, 128)          598912      ['board_input[0][0]']            
                                                                                      

In [18]:
train_model(model, n_files=5)

Training on file chess1.csv
Processing...complete
14063/14063 [==============================] - 226s 16ms/step - loss: 5.0435 - sparse_categorical_accuracy: 0.1156 - val_loss: 4.4483 - val_sparse_categorical_accuracy: 0.1589
Training on file chess2.csv
Processing...complete
14063/14063 [==============================] - 219s 16ms/step - loss: 4.2929 - sparse_categorical_accuracy: 0.1640 - val_loss: 4.2190 - val_sparse_categorical_accuracy: 0.1660
Training on file chess3.csv
Processing...complete
14063/14063 [==============================] - 221s 16ms/step - loss: 4.1036 - sparse_categorical_accuracy: 0.1725 - val_loss: 4.0813 - val_sparse_categorical_accuracy: 0.1690
Training on file chess4.csv
Processing...complete
14063/14063 [==============================] - 221s 16ms/step - loss: 3.9518 - sparse_categorical_accuracy: 0.1824 - val_loss: 3.8847 - val_sparse_categorical_accuracy: 0.1868
Training on file chess5.csv
Processing...complete
14063/14063 [==============================] -

In [26]:
# model conv2d (semi) seq expanded conv output
activation = "relu"

inputs = keras.Input(shape=(70), name="all_input")

input1 = keras.layers.Lambda(lambda x: tf.slice(x, (0, 0), (-1, 1)), name="player_input")(inputs)
input2 = keras.layers.Lambda(lambda x: tf.slice(x, (0, 1), (-1, 64)), name="board_input")(inputs)
input3 = keras.layers.Lambda(lambda x: tf.slice(x, (0, 65), (-1, 5)), name="misc_input")(inputs)

conv_model = tf.keras.models.Sequential([  # 1:65
    keras.layers.Reshape((8, 8, 1), input_shape=(64,)),
    keras.layers.Conv2D(64, 3, padding="same", activation=activation),
    keras.layers.Conv2D(64, 3, padding="same", activation=activation),
    keras.layers.Conv2D(64, 3, padding="same", activation=activation),
    keras.layers.Flatten(),
    keras.layers.Dense(192)
], name="conv_model")
conv_output = conv_model(input2)

dense_inputs = keras.layers.concatenate([input1, conv_output, input3])
dense_model = tf.keras.models.Sequential([  # tf.concat([input[:1], conv_model_output, input[65:]], axis=0)
    keras.layers.Input(198),
    keras.layers.Dense(256, activation=activation),
    keras.layers.Dense(256, activation=activation),
    keras.layers.Dense(constants.LEN_UCI_MOVES, activation="softmax")
], name="dense_model")
dense_output = dense_model(dense_inputs)

model = tf.keras.models.Model(inputs, dense_output, name="conv2d_seq_cx")
model.summary()

model.compile(
    optimizer = keras.optimizers.Adam(learning_rate=0.001),
    loss = keras.losses.SparseCategoricalCrossentropy(),
    metrics = [keras.metrics.SparseCategoricalAccuracy()],
)
train_model(model, n_files=5)

Model: "conv2d_seq_cx"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 all_input (InputLayer)         [(None, 70)]         0           []                               
                                                                                                  
 board_input (Lambda)           (None, 64)           0           ['all_input[0][0]']              
                                                                                                  
 player_input (Lambda)          (None, 1)            0           ['all_input[0][0]']              
                                                                                                  
 conv_model (Sequential)        (None, 192)          861120      ['board_input[0][0]']            
                                                                                      

In [34]:
# model conv2d (semi) sequential with expanded conv 
activation = "relu"

inputs = keras.Input(shape=(70), name="all_input")

input1 = keras.layers.Lambda(lambda x: tf.slice(x, (0, 0), (-1, 1)), name="player_input")(inputs)
input2 = keras.layers.Lambda(lambda x: tf.slice(x, (0, 1), (-1, 64)), name="board_input")(inputs)
input3 = keras.layers.Lambda(lambda x: tf.slice(x, (0, 65), (-1, 5)), name="misc_input")(inputs)

conv_model = tf.keras.models.Sequential([  # 1:65
    keras.layers.Reshape((8, 8, 1), input_shape=(64,)),
    keras.layers.Conv2D(128, 3, padding="same", activation=activation),
    keras.layers.Conv2D(128, 3, padding="same", activation=activation),
    keras.layers.Conv2D(128, 3, padding="same", activation=activation),
    keras.layers.Flatten(),
    keras.layers.Dense(128)
], name="conv_model")
conv_output = conv_model(input2)

dense_inputs = keras.layers.concatenate([input1, conv_output, input3])
dense_model = tf.keras.models.Sequential([  # tf.concat([input[:1], conv_model_output, input[65:]], axis=0)
    keras.layers.Input(134),
    keras.layers.Dense(256, activation=activation),
    keras.layers.Dense(256, activation=activation),
    keras.layers.Dense(constants.LEN_UCI_MOVES, activation="softmax")
], name="dense_model")
dense_output = dense_model(dense_inputs)

model = tf.keras.models.Model(inputs, dense_output, name="conv2d_seq_cx2")
model.summary()

model.compile(
    optimizer = keras.optimizers.Adam(learning_rate=0.001),
    loss = keras.losses.SparseCategoricalCrossentropy(),
    metrics = [keras.metrics.SparseCategoricalAccuracy()],
)
train_model(model, n_files=5)

Model: "conv2d_seq_cx2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 all_input (InputLayer)         [(None, 70)]         0           []                               
                                                                                                  
 board_input (Lambda)           (None, 64)           0           ['all_input[0][0]']              
                                                                                                  
 player_input (Lambda)          (None, 1)            0           ['all_input[0][0]']              
                                                                                                  
 conv_model (Sequential)        (None, 128)          1345152     ['board_input[0][0]']            
                                                                                     

In [30]:
# model conv2d (semi) sequential with expanded conv reduced dense
activation = "relu"

inputs = keras.Input(shape=(70), name="all_input")

input1 = keras.layers.Lambda(lambda x: tf.slice(x, (0, 0), (-1, 1)), name="player_input")(inputs)
input2 = keras.layers.Lambda(lambda x: tf.slice(x, (0, 1), (-1, 64)), name="board_input")(inputs)
input3 = keras.layers.Lambda(lambda x: tf.slice(x, (0, 65), (-1, 5)), name="misc_input")(inputs)

conv_model = tf.keras.models.Sequential([  # 1:65
    keras.layers.Reshape((8, 8, 1), input_shape=(64,)),
    keras.layers.Conv2D(64, 3, padding="same", activation=activation),
    keras.layers.Conv2D(64, 3, padding="same", activation=activation),
    keras.layers.Conv2D(64, 3, padding="same", activation=activation),
    keras.layers.Flatten(),
    keras.layers.Dense(192)
], name="conv_model")
conv_output = conv_model(input2)

dense_inputs = keras.layers.concatenate([input1, conv_output, input3])
dense_model = tf.keras.models.Sequential([  # tf.concat([input[:1], conv_model_output, input[65:]], axis=0)
    keras.layers.Input(198),
    keras.layers.Dense(256, activation=activation),
    keras.layers.Dense(constants.LEN_UCI_MOVES, activation="softmax")
], name="dense_model")
dense_output = dense_model(dense_inputs)

model = tf.keras.models.Model(inputs, dense_output, name="conv2d_seq_cx_dr")
model.summary()

model.compile(
    optimizer = keras.optimizers.Adam(learning_rate=0.001),
    loss = keras.losses.SparseCategoricalCrossentropy(),
    metrics = [keras.metrics.SparseCategoricalAccuracy()],
)
train_model(model, n_files=5)

Model: "conv2d_seq_cx_dr"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 all_input (InputLayer)         [(None, 70)]         0           []                               
                                                                                                  
 board_input (Lambda)           (None, 64)           0           ['all_input[0][0]']              
                                                                                                  
 player_input (Lambda)          (None, 1)            0           ['all_input[0][0]']              
                                                                                                  
 conv_model (Sequential)        (None, 192)          861120      ['board_input[0][0]']            
                                                                                   

In [32]:
# model conv2d (semi) sequential with expanded conv reduced dense
activation = "relu"

inputs = keras.Input(shape=(70), name="all_input")

input1 = keras.layers.Lambda(lambda x: tf.slice(x, (0, 0), (-1, 1)), name="player_input")(inputs)
input2 = keras.layers.Lambda(lambda x: tf.slice(x, (0, 1), (-1, 64)), name="board_input")(inputs)
input3 = keras.layers.Lambda(lambda x: tf.slice(x, (0, 65), (-1, 5)), name="misc_input")(inputs)

conv_model = tf.keras.models.Sequential([  # 1:65
    keras.layers.Reshape((8, 8, 1), input_shape=(64,)),
    keras.layers.Conv2D(64, 3, padding="same", activation=activation),
    keras.layers.Conv2D(64, 3, padding="same", activation=activation),
    keras.layers.Conv2D(64, 3, padding="same", activation=activation),
    keras.layers.Flatten(),
    keras.layers.Dense(192)
], name="conv_model")
conv_output = conv_model(input2)

dense_inputs = keras.layers.concatenate([input1, conv_output, input3])
dense_model = tf.keras.models.Sequential([  # tf.concat([input[:1], conv_model_output, input[65:]], axis=0)
    keras.layers.Input(198),
    keras.layers.Dense(constants.LEN_UCI_MOVES, activation="softmax")
], name="dense_model")
dense_output = dense_model(dense_inputs)

model = tf.keras.models.Model(inputs, dense_output, name="conv2d_seq_cx_dr2")
model.summary()

model.compile(
    optimizer = keras.optimizers.Adam(learning_rate=0.001),
    loss = keras.losses.SparseCategoricalCrossentropy(),
    metrics = [keras.metrics.SparseCategoricalAccuracy()],
)
train_model(model, n_files=5)

Model: "conv2d_seq_cx_dr2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 all_input (InputLayer)         [(None, 70)]         0           []                               
                                                                                                  
 board_input (Lambda)           (None, 64)           0           ['all_input[0][0]']              
                                                                                                  
 player_input (Lambda)          (None, 1)            0           ['all_input[0][0]']              
                                                                                                  
 conv_model (Sequential)        (None, 192)          861120      ['board_input[0][0]']            
                                                                                  

In [34]:
# model conv2d (semi) sequential with expanded conv reduced dense
activation = "relu"

inputs = keras.Input(shape=(70), name="all_input")

input1 = keras.layers.Lambda(lambda x: tf.slice(x, (0, 0), (-1, 1)), name="player_input")(inputs)
input2 = keras.layers.Lambda(lambda x: tf.slice(x, (0, 1), (-1, 64)), name="board_input")(inputs)
input3 = keras.layers.Lambda(lambda x: tf.slice(x, (0, 65), (-1, 5)), name="misc_input")(inputs)

conv_model = tf.keras.models.Sequential([  # 1:65
    keras.layers.Reshape((8, 8, 1), input_shape=(64,)),
    keras.layers.Conv2D(64, 3, padding="same", activation=activation),
    keras.layers.Conv2D(64, 3, padding="same", activation=activation),
    # keras.layers.Conv2D(64, 3, padding="same", activation=activation),
    keras.layers.Flatten(),
    keras.layers.Dense(192)
], name="conv_model")
conv_output = conv_model(input2)

dense_inputs = keras.layers.concatenate([input1, conv_output, input3])
dense_model = tf.keras.models.Sequential([  # tf.concat([input[:1], conv_model_output, input[65:]], axis=0)
    keras.layers.Input(198),
    keras.layers.Dense(constants.LEN_UCI_MOVES, activation="softmax")
], name="dense_model")
dense_output = dense_model(dense_inputs)

model = tf.keras.models.Model(inputs, dense_output, name="conv2d_seq_cx3_dr2")
model.summary()

model.compile(
    optimizer = keras.optimizers.Adam(learning_rate=0.001),
    loss = keras.losses.SparseCategoricalCrossentropy(),
    metrics = [keras.metrics.SparseCategoricalAccuracy()],
)
train_model(model, n_files=5)

Model: "conv2d_seq_cx3_dr2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 all_input (InputLayer)         [(None, 70)]         0           []                               
                                                                                                  
 board_input (Lambda)           (None, 64)           0           ['all_input[0][0]']              
                                                                                                  
 player_input (Lambda)          (None, 1)            0           ['all_input[0][0]']              
                                                                                                  
 conv_model (Sequential)        (None, 192)          824192      ['board_input[0][0]']            
                                                                                 

In [39]:
# model conv2d (semi) sequential with expanded conv reduced dense
activation = "relu"

inputs = keras.Input(shape=(70), name="all_input")

input1 = keras.layers.Lambda(lambda x: tf.slice(x, (0, 0), (-1, 1)), name="player_input")(inputs)
input2 = keras.layers.Lambda(lambda x: tf.slice(x, (0, 1), (-1, 64)), name="board_input")(inputs)
input3 = keras.layers.Lambda(lambda x: tf.slice(x, (0, 65), (-1, 5)), name="misc_input")(inputs)

conv_model = tf.keras.models.Sequential([  # 1:65
    keras.layers.Reshape((8, 8, 1), input_shape=(64,)),
    keras.layers.Conv2D(64, 3, padding="same", activation=activation),
    keras.layers.Conv2D(64, 3, padding="same", activation=activation),
    keras.layers.Conv2D(64, 3, padding="same", activation=activation),
    keras.layers.Flatten(),
    keras.layers.Dense(256)
], name="conv_model")
conv_output = conv_model(input2)

dense_inputs = keras.layers.concatenate([input1, conv_output, input3])
dense_model = tf.keras.models.Sequential([  # tf.concat([input[:1], conv_model_output, input[65:]], axis=0)
    keras.layers.Input(262),
    keras.layers.Dense(constants.LEN_UCI_MOVES, activation="softmax")
], name="dense_model")
dense_output = dense_model(dense_inputs)

model = tf.keras.models.Model(inputs, dense_output, name="conv2d_seq_cxx_dr2")
model.summary()

model.compile(
    optimizer = keras.optimizers.Adam(learning_rate=0.001),
    loss = keras.losses.SparseCategoricalCrossentropy(),
    metrics = [keras.metrics.SparseCategoricalAccuracy()],
)
train_model(model, n_files=5)

Model: "conv2d_seq_cxx_dr2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 all_input (InputLayer)         [(None, 70)]         0           []                               
                                                                                                  
 board_input (Lambda)           (None, 64)           0           ['all_input[0][0]']              
                                                                                                  
 player_input (Lambda)          (None, 1)            0           ['all_input[0][0]']              
                                                                                                  
 conv_model (Sequential)        (None, 256)          1123328     ['board_input[0][0]']            
                                                                                 

In [42]:
# model conv2d (semi) sequential with expanded conv reduced dense
activation = "relu"

inputs = keras.Input(shape=(70), name="all_input")

input1 = keras.layers.Lambda(lambda x: tf.slice(x, (0, 0), (-1, 1)), name="player_input")(inputs)
input2 = keras.layers.Lambda(lambda x: tf.slice(x, (0, 1), (-1, 64)), name="board_input")(inputs)
input3 = keras.layers.Lambda(lambda x: tf.slice(x, (0, 65), (-1, 5)), name="misc_input")(inputs)

conv_model = tf.keras.models.Sequential([  # 1:65
    keras.layers.Reshape((8, 8, 1), input_shape=(64,)),
    keras.layers.Conv2D(64, 3, padding="same", activation=activation),
    keras.layers.Conv2D(64, 3, padding="same", activation=activation),
    keras.layers.Conv2D(64, 3, padding="same", activation=activation),
    keras.layers.Conv2D(64, 3, padding="same", activation=activation),
    keras.layers.Flatten(),
    keras.layers.Dense(256)
], name="conv_model")
conv_output = conv_model(input2)

dense_inputs = keras.layers.concatenate([input1, conv_output, input3])
dense_model = tf.keras.models.Sequential([  # tf.concat([input[:1], conv_model_output, input[65:]], axis=0)
    keras.layers.Input(262),
    keras.layers.Dense(constants.LEN_UCI_MOVES, activation="softmax")
], name="dense_model")
dense_output = dense_model(dense_inputs)

model = tf.keras.models.Model(inputs, dense_output, name="conv2d_seq_c4_d0")
model.summary()

model.compile(
    optimizer = keras.optimizers.Adam(learning_rate=0.001),
    loss = keras.losses.SparseCategoricalCrossentropy(),
    metrics = [keras.metrics.SparseCategoricalAccuracy()],
)
train_model(model, n_files=5)

Model: "conv2d_seq_c4_d0"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 all_input (InputLayer)         [(None, 70)]         0           []                               
                                                                                                  
 board_input (Lambda)           (None, 64)           0           ['all_input[0][0]']              
                                                                                                  
 player_input (Lambda)          (None, 1)            0           ['all_input[0][0]']              
                                                                                                  
 conv_model (Sequential)        (None, 256)          1160256     ['board_input[0][0]']            
                                                                                   

In [44]:
# model conv2d (semi) sequential with expanded conv reduced dense
activation = "relu"

inputs = keras.Input(shape=(70), name="all_input")

input1 = keras.layers.Lambda(lambda x: tf.slice(x, (0, 0), (-1, 1)), name="player_input")(inputs)
input2 = keras.layers.Lambda(lambda x: tf.slice(x, (0, 1), (-1, 64)), name="board_input")(inputs)
input3 = keras.layers.Lambda(lambda x: tf.slice(x, (0, 65), (-1, 5)), name="misc_input")(inputs)

conv_model = tf.keras.models.Sequential([  # 1:65
    keras.layers.Reshape((8, 8, 1), input_shape=(64,)),
    keras.layers.Conv2D(64, 3, padding="same", activation=activation),
    keras.layers.Conv2D(64, 3, padding="same", activation=activation),
    keras.layers.Conv2D(64, 3, padding="same", activation=activation),
    keras.layers.Conv2D(64, 3, padding="same", activation=activation),
    keras.layers.Conv2D(64, 3, padding="same", activation=activation),
    keras.layers.Flatten(),
    keras.layers.Dense(256)
], name="conv_model")
conv_output = conv_model(input2)

dense_inputs = keras.layers.concatenate([input1, conv_output, input3])
dense_model = tf.keras.models.Sequential([  # tf.concat([input[:1], conv_model_output, input[65:]], axis=0)
    keras.layers.Input(262),
    keras.layers.Dense(constants.LEN_UCI_MOVES, activation="softmax")
], name="dense_model")
dense_output = dense_model(dense_inputs)

model = tf.keras.models.Model(inputs, dense_output, name="conv2d_seq_c5_d0")
model.summary()

model.compile(
    optimizer = keras.optimizers.Adam(learning_rate=0.001),
    loss = keras.losses.SparseCategoricalCrossentropy(),
    metrics = [keras.metrics.SparseCategoricalAccuracy()],
)
train_model(model, n_files=5)

Model: "conv2d_seq_c5_d0"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 all_input (InputLayer)         [(None, 70)]         0           []                               
                                                                                                  
 board_input (Lambda)           (None, 64)           0           ['all_input[0][0]']              
                                                                                                  
 player_input (Lambda)          (None, 1)            0           ['all_input[0][0]']              
                                                                                                  
 conv_model (Sequential)        (None, 256)          1197184     ['board_input[0][0]']            
                                                                                   

In [45]:
models = ["conv2d_seq_cx_dr", "conv2d_seq_cx_dr2", "conv2d_seq_cxx_dr2", "conv2d_seq_c4_d0", "conv2d_seq_c5_d0"]
results = evaluate_model(models)
results

Restoring from ckpt/conv2d_seq_cx_dr/conv2d_seq_cx_dr-5.h5
Restoring from ckpt/conv2d_seq_cx_dr2/conv2d_seq_cx_dr2-5.h5
Restoring from ckpt/conv2d_seq_cxx_dr2/conv2d_seq_cxx_dr2-5.h5
Restoring from ckpt/conv2d_seq_c4_d0/conv2d_seq_c4_d0-5.h5
Restoring from ckpt/conv2d_seq_c5_d0/conv2d_seq_c5_d0-5.h5
Processing...complete
5451/5451 [==============================] - 69s 13ms/step - loss: 3.1406 - sparse_categorical_accuracy: 0.2478


,model_name,sparse_categorical_accuracy
0,conv2d_seq_cx_dr,0.236640
1,conv2d_seq_cx_dr2,0.240458
2,conv2d_seq_cxx_dr2,0.244143
3,conv2d_seq_c4_d0,0.244680
4,conv2d_seq_c5_d0,0.247849


In [16]:
import chess.pgn

file = "data/test.pgn"
with open(file, "r") as f:
    game = chess.pgn.read_game(f)
board = game.board()
for move in game.mainline_moves():
    board.push(move)
print(board)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
print(board.outcome())

. . k r . . . .
p p . . . p p p
. . p . . . . .
. . . . p . . n
. . . . P . . .
. . N n . P . P
P . . r B . P .
. R R . . K . .
None


In [12]:
end = game.end()
end.uci()

'f2f1'

In [18]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("distribution.csv")
df.head()

,turn,state,action,visits,reward,ucb
0,0,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,g1h3,375,172.600000,0.769440
1,0,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,g1f3,404,190.666667,0.769817
2,0,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,b1c3,399,187.516667,0.769697
3,0,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,b1a3,375,172.666667,0.769617
4,0,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,h2h3,393,183.733333,0.769524


In [ ]:
for turn, d in df.groupby(["turn"]):
    print(turn)
    ds = d.sort_values("visits", ascending=False)
    plt.plot("action", "visits", data=ds)
    plt.show()